In [2]:
#importing necessary modules
import pandas as pd
import plotly.express as px
import pymssql
from shapely import wkb
import geopandas as gpd


In [3]:
#setting database connection strings
database = 'Pushing-P-DB'
table = 'dbo.NJ_Cities'
user = 'pushing_p'
password  = 't3stP@ssword'
server = 'gen10-data-fundamentals-22-02-sql-server.database.windows.net'

In [5]:
# Retrieve Training Data for Geographically Weighted Regression Model
conn = pymssql.connect(server, user, password, database)

# Query select all rows from SQL tables to insert into their respective DataFrames
query1 = f'SELECT * FROM {table}'

cities = pd.read_sql(query1, conn)

# Function convert hex string to polygon objects
def convert_to_polygon(hex):
    return wkb.loads(hex, hex=True)
cities['geometry'] = cities['geometry'].apply(convert_to_polygon)


# Convert from pandas to geopandas dataframes
df = gpd.GeoDataFrame(cities, geometry=cities['geometry'], crs = 'epsg:4269')

#converting Number of Uninsured column to intiger
# df['Number of Uninsured (2019)'] = df['Number of Uninsured (2019)'].astype('int64')

c:\Users\Luis\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [6]:
cities.head(2)

,State,County,City,Unemployment Rate (16 & Over),Median Household Income,City Population,Uninsured Population,"NUI, Under 6 years","NUI, 6-18 years","NUI, 19 to 25 years",...,"NUI, 65 years and older","NUI, Men","NUI, Women","NUI, White","NUI, African American","NUI, American Indian & Alaska Native Population","NUI, Asians","NUI, Native Hawaiians & Pacific Islanders Population","NUI, Other Races",geometry
0,New Jersey,Camden County,Brooklawn Borough,6.3,44805,1888,95,0,15,18,...,0,49,46,87,0,0,0,0,0,"POLYGON ((-75.13175 39.87852, -75.12834 39.879..."
1,New Jersey,Middlesex County,Metuchen Borough,4.4,136067,14289,300,0,10,50,...,10,187,113,65,58,0,157,0,20,"POLYGON ((-74.38258 40.55220, -74.38032 40.552..."


In [18]:
# 1 Bar Graph

# Here I set the colors I will be using for the visualizations
colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}


df1= df[['City','Uninsured Population']]

df1 = df1.sort_values(by='Uninsured Population', ascending =False)

df1 = df1.head(10)

fig_1 = px.bar(df1, x='City', y='Uninsured Population',color_discrete_sequence=["blue"], title='Uninsured Population Of Top 10 Cities', height = 500, width = 700)
fig_1.update_layout(xaxis={'categoryorder':'total descending'})
# fig_3.update_layout(width= 1150, height=700)
# fig_3.show()

# this is where I center the title for the visualization
fig_1.update_layout(
    title=dict(x=0.7), #set title in the center
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text'],
)


In [57]:
# 3 Treemap

# Here I set the colors I will be using for the visualizations
colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}


# Here I set the colors I will be using for the visualizations
colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}

df3= cities[['NUI, White', 'NUI, African American','NUI, American Indian & Alaska Native Population',
'NUI, Asians','NUI, Native Hawaiians & Pacific Islanders Population', 'NUI, Other Races','Uninsured Population']]

df3.rename(columns={"NUI, American Indian & Alaska Native Population": "NUI, American Indian &\n Alaska Native", "NUI, Native Hawaiians & Pacific Islanders Population": "NUI, Native Hawaiians &\n Pacific Islanders"}, inplace = True)

df3 = df3[['NUI, White', 'NUI, African American','NUI, American Indian &\n Alaska Native',
'NUI, Asians','NUI, Native Hawaiians &\n Pacific Islanders', 'NUI, Other Races']].sum()

df3 = pd.DataFrame(df3)

df3 = df3.reset_index()

# Didnt use an equals sign because this just renames the existing dataframe instead of renaming a copy of the dataframe.
df3.rename(columns = {'index':'Race', 0:'Number Of Uninsured'}, inplace = True)

df3 = df3.sort_values(by='Number Of Uninsured', ascending =False)

# fig_3 = px.treemap(df3, path=['Race'], values='Number Of Uninsured')

fig_3 = px.treemap(df3, path=['Race'],
                 values='Number Of Uninsured',
                 color='Number Of Uninsured')

fig_3.update_traces(root_color="black")
fig_3.data[0]['textfont']['color'] = colors['text']
fig_3.data[0].hovertemplate = '%{label}<br>%{value}'
fig_3.update_layout(
    title= "Number Of Uninsured By Race in New Jersey"
)

# this is where I center the title for the visualization
fig_3.update_layout(
    height = 700, width = 900,
    title=dict(x=0.5), #set title in the center
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text'],
)



C:\Users\Luis\AppData\Local\Temp\ipykernel_12908\4094274395.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\Luis\anaconda3\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [30]:
# 4 Horizontal Bar

# Here I set the colors I will be using for the visualizations
colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}

df5= cities[['NUI, White', 'NUI, African American','NUI, American Indian & Alaska Native Population',
'NUI, Asians','NUI, Native Hawaiians & Pacific Islanders Population', 'NUI, Other Races','Uninsured Population']]

df5.rename(columns={"NUI, American Indian & Alaska Native Population": "NUI, American Indian &\n Alaska Native", "NUI, Native Hawaiians & Pacific Islanders Population": "NUI, Native Hawaiians &\n Pacific Islanders"}, inplace = True)

df5 = df5[['NUI, White', 'NUI, African American','NUI, American Indian &\n Alaska Native',
'NUI, Asians','NUI, Native Hawaiians &\n Pacific Islanders', 'NUI, Other Races']].sum()

df5 = pd.DataFrame(df5)

df5 = df5.reset_index()

# Didnt use an equals sign because this just renames the existing dataframe instead of renaming a copy of the dataframe.
df5.rename(columns = {'index':'Race', 0:'Number Of Uninsured'}, inplace = True)

df5 = df5.sort_values(by='Number Of Uninsured', ascending =True)

fig_5 = px.bar(df5, x='Number Of Uninsured', y='Race',color_discrete_sequence=["blue"], title='Number Of Uninsured By Race For New Jersey', height = 500, width = 700, orientation= 'h')
# fig_3.update_layout(width= 1150, height=700)
# fig_3.show()

# this is where I center the title for the visualization
fig_5.update_layout(
    title=dict(x=0.7), #set title in the center
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text'],
)

fig_5.show()


C:\Users\Luis\AppData\Local\Temp\ipykernel_12908\780398220.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

